In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import re
import json
from google.colab import files
import io
from typing import Dict, List, Optional, Union
import logging
from dataclasses import dataclass
import hashlib
import matplotlib.pyplot as plt

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

@dataclass
class EmployeeData:
    """Data class for employee information validation"""
    employee_id: Optional[int] = None
    first_name: str = ''
    last_name: str = ''
    email: str = ''
    department: str = ''
    position: str = ''
    hire_date: str = ''
    salary: float = 0.0
    manager: str = ''
    status: str = 'Active'

class DataValidator:
    """Handles all data validation for the HR system"""

    @staticmethod
    def validate_email(email: str) -> bool:
        """Validate email format"""
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(pattern, email))

    @staticmethod
    def validate_date(date_str: str) -> bool:
        """Validate date format (YYYY-MM-DD)"""
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            return True
        except ValueError:
            return False

    @staticmethod
    def validate_salary(salary: float) -> bool:
        """Validate salary (must be positive)"""
        return isinstance(salary, (int, float)) and salary > 0

    @staticmethod
    def validate_employee_data(data: Dict) -> tuple[bool, str]:
        """Validate all employee data fields"""
        required_fields = ['first_name', 'last_name', 'email', 'department',
                         'position', 'hire_date', 'salary', 'manager']

        # Check required fields
        for field in required_fields:
            if field not in data:
                return False, f"Missing required field: {field}"

        # Validate email
        if not DataValidator.validate_email(data['email']):
            return False, "Invalid email format"

        # Validate hire date
        if not DataValidator.validate_date(data['hire_date']):
            return False, "Invalid hire date format (use YYYY-MM-DD)"

        # Validate salary
        if not DataValidator.validate_salary(data['salary']):
            return False, "Invalid salary (must be positive number)"

        return True, "Validation successful"

class DatabaseManager:
    """Handles all database operations"""

    def __init__(self, data_dir: str = 'hr_data'):
        self.data_dir = data_dir
        self.create_data_directory()

    def create_data_directory(self):
        """Create directory for storing data if it doesn't exist"""
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)

    def save_dataframe(self, df: pd.DataFrame, filename: str):
        """Save DataFrame to CSV"""
        filepath = os.path.join(self.data_dir, f"{filename}.csv")
        df.to_csv(filepath, index=False)

    def load_dataframe(self, filename: str) -> pd.DataFrame:
        """Load DataFrame from CSV"""
        filepath = os.path.join(self.data_dir, f"{filename}.csv")
        if os.path.exists(filepath):
            return pd.read_csv(filepath)
        return pd.DataFrame()

class SecurityManager:
    """Handles security-related operations"""

    @staticmethod
    def hash_password(password: str) -> str:
        """Hash password using SHA-256"""
        return hashlib.sha256(password.encode()).hexdigest()

    @staticmethod
    def generate_employee_id() -> str:
        """Generate unique employee ID"""
        return datetime.now().strftime('%Y%m%d%H%M%S')

class HRAutomationSystem:
    """Main HR Automation System class"""

    def __init__(self):
        self.db_manager = DatabaseManager()
        self.validator = DataValidator()
        self.security = SecurityManager()

        # Initialize DataFrames
        self.employees_db = pd.DataFrame(columns=[
            'employee_id', 'first_name', 'last_name', 'email',
            'department', 'position', 'hire_date', 'salary',
            'manager', 'status', 'last_updated'
        ])

        self.leave_requests = pd.DataFrame(columns=[
            'request_id', 'employee_id', 'leave_type',
            'start_date', 'end_date', 'status', 'comments',
            'approved_by', 'approved_date'
        ])

        self.attendance_records = pd.DataFrame(columns=[
            'record_id', 'employee_id', 'date', 'check_in',
            'check_out', 'status', 'working_hours', 'overtime'
        ])

        self.performance_reviews = pd.DataFrame(columns=[
            'review_id', 'employee_id', 'review_date', 'reviewer',
            'performance_score', 'strengths', 'areas_for_improvement',
            'goals', 'comments'
        ])

        self.training_records = pd.DataFrame(columns=[
            'training_id', 'employee_id', 'training_name',
            'start_date', 'end_date', 'status', 'score',
            'certificate_id'
        ])

        # Load existing data if available
        self._load_data()

    def _load_data(self):
        """Load all data from storage"""
        try:
            self.employees_db = self.db_manager.load_dataframe('employees')
            self.leave_requests = self.db_manager.load_dataframe('leave_requests')
            self.attendance_records = self.db_manager.load_dataframe('attendance')
            self.performance_reviews = self.db_manager.load_dataframe('performance')
            self.training_records = self.db_manager.load_dataframe('training')
        except Exception as e:
            logger.error(f"Error loading data: {str(e)}")

    def _save_data(self):
        """Save all data to storage"""
        try:
            self.db_manager.save_dataframe(self.employees_db, 'employees')
            self.db_manager.save_dataframe(self.leave_requests, 'leave_requests')
            self.db_manager.save_dataframe(self.attendance_records, 'attendance')
            self.db_manager.save_dataframe(self.performance_reviews, 'performance')
            self.db_manager.save_dataframe(self.training_records, 'training')
        except Exception as e:
            logger.error(f"Error saving data: {str(e)}")

    def add_employee(self, emp_data: Dict) -> tuple[bool, Union[int, str]]:
        """Add a new employee to the system"""
        try:
            # Validate employee data
            is_valid, message = self.validator.validate_employee_data(emp_data)
            if not is_valid:
                return False, message

            # Generate employee ID
            emp_data['employee_id'] = self.security.generate_employee_id()
            emp_data['status'] = 'Active'
            emp_data['last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # Add to database
            self.employees_db = pd.concat([
                self.employees_db,
                pd.DataFrame([emp_data])
            ], ignore_index=True)

            self._save_data()
            logger.info(f"Added new employee: {emp_data['first_name']} {emp_data['last_name']}")

            return True, emp_data['employee_id']

        except Exception as e:
            logger.error(f"Error adding employee: {str(e)}")
            return False, str(e)

    def update_employee(self, emp_id: str, update_data: Dict) -> tuple[bool, str]:
        """Update employee information"""
        try:
            if emp_id not in self.employees_db['employee_id'].values:
                return False, "Employee not found"

            # Validate update data
            if 'email' in update_data:
                if not self.validator.validate_email(update_data['email']):
                    return False, "Invalid email format"

            if 'salary' in update_data:
                if not self.validator.validate_salary(update_data['salary']):
                    return False, "Invalid salary"

            # Update employee data
            idx = self.employees_db[self.employees_db['employee_id'] == emp_id].index[0]
            for key, value in update_data.items():
                self.employees_db.at[idx, key] = value

            self.employees_db.at[idx, 'last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            self._save_data()
            return True, "Employee updated successfully"

        except Exception as e:
            logger.error(f"Error updating employee: {str(e)}")
            return False, str(e)

    def process_leave_request(self, request_data: Dict) -> tuple[bool, Union[int, str]]:
        """Process a leave request"""
        try:
            # Validate dates
            if not (self.validator.validate_date(request_data['start_date']) and
                   self.validator.validate_date(request_data['end_date'])):
                return False, "Invalid date format"

            # Check if employee exists
            if request_data['employee_id'] not in self.employees_db['employee_id'].values:
                return False, "Employee not found"

            request_id = len(self.leave_requests) + 1
            request_data['request_id'] = request_id
            request_data['status'] = 'Pending'

            self.leave_requests = pd.concat([
                self.leave_requests,
                pd.DataFrame([request_data])
            ], ignore_index=True)

            self._save_data()
            return True, request_id

        except Exception as e:
            logger.error(f"Error processing leave request: {str(e)}")
            return False, str(e)

    def record_attendance(self, emp_id: str, check_in: str = None,
                        check_out: str = None) -> tuple[bool, str]:
        """Record employee attendance"""
        try:
            if emp_id not in self.employees_db['employee_id'].values:
                return False, "Employee not found"

            current_time = datetime.now()
            check_in = check_in or current_time.strftime('%H:%M:%S')

            record = {
                'record_id': len(self.attendance_records) + 1,
                'employee_id': emp_id,
                'date': current_time.date().strftime('%Y-%m-%d'),
                'check_in': check_in,
                'check_out': check_out,
                'status': 'Present'
            }

            # Calculate working hours if check-out is provided
            if check_out:
                check_in_time = datetime.strptime(check_in, '%H:%M:%S')
                check_out_time = datetime.strptime(check_out, '%H:%M:%S')
                working_hours = (check_out_time - check_in_time).total_seconds() / 3600

                record['working_hours'] = round(working_hours, 2)
                record['overtime'] = max(0, working_hours - 8)  # Assuming 8-hour workday

            self.attendance_records = pd.concat([
                self.attendance_records,
                pd.DataFrame([record])
            ], ignore_index=True)

            self._save_data()
            return True, "Attendance recorded successfully"

        except Exception as e:
            logger.error(f"Error recording attendance: {str(e)}")
            return False, str(e)

    def generate_payroll_report(self, month: int, year: int) -> pd.DataFrame:
        """Generate monthly payroll report"""
        try:
            active_employees = self.employees_db[self.employees_db['status'] == 'Active']
            payroll_data = []

            for _, emp in active_employees.iterrows():
                # Calculate working days
                month_attendance = self.attendance_records[
                    (self.attendance_records['employee_id'] == emp['employee_id']) &
                    (pd.to_datetime(self.attendance_records['date']).dt.month == month) &
                    (pd.to_datetime(self.attendance_records['date']).dt.year == year)
                ]

                working_days = len(month_attendance)
                total_hours = month_attendance['working_hours'].sum()
                overtime_hours = month_attendance['overtime'].sum()

                # Calculate leave days
                month_leaves = self.leave_requests[
                    (self.leave_requests['employee_id'] == emp['employee_id']) &
                    (self.leave_requests['status'] == 'Approved') &
                    (pd.to_datetime(self.leave_requests['start_date']).dt.month == month) &
                    (pd.to_datetime(self.leave_requests['start_date']).dt.year == year)
                ]

                leave_days = sum(
                    (pd.to_datetime(leave['end_date']) - pd.to_datetime(leave['start_date'])).days + 1
                    for _, leave in month_leaves.iterrows()
                )

                # Calculate salary components
                daily_salary = emp['salary'] / 22  # Assuming 22 working days per month
                base_salary = daily_salary * (working_days - leave_days)
                overtime_rate = daily_salary / 8 * 1.5  # 1.5x overtime rate
                overtime_pay = overtime_hours * overtime_rate

                payroll_data.append({
                    'employee_id': emp['employee_id'],
                    'name': f"{emp['first_name']} {emp['last_name']}",
                    'department': emp['department'],
                    'working_days': working_days,
                    'leave_days': leave_days,
                    'total_hours': total_hours,
                    'overtime_hours': overtime_hours,
                    'base_salary': base_salary,
                    'overtime_pay': overtime_pay,
                    'gross_salary': base_salary + overtime_pay,
                    'deductions': (base_salary + overtime_pay) * 0.1,  # Assuming 10% deductions
                    'net_salary': (base_salary + overtime_pay) * 0.9
                })

            return pd.DataFrame(payroll_data)

        except Exception as e:
            logger.error(f"Error generating payroll report: {str(e)}")
            return pd.DataFrame()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import re
import json
from google.colab import files
import io
from typing import Dict, List, Optional, Union
import logging
from dataclasses import dataclass
import hashlib
import matplotlib.pyplot as plt

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class HRAutomationSystem:
    def __init__(self):
        # Initialize DataFrames
        self.employees_db = pd.DataFrame(columns=[
            'employee_id', 'first_name', 'last_name', 'email',
            'department', 'position', 'hire_date', 'salary',
            'manager', 'status', 'last_updated'
        ])

        self.leave_requests = pd.DataFrame(columns=[
            'request_id', 'employee_id', 'leave_type',
            'start_date', 'end_date', 'status', 'comments',
            'approved_by', 'approved_date'
        ])

        self.attendance_records = pd.DataFrame(columns=[
            'record_id', 'employee_id', 'date', 'check_in',
            'check_out', 'status', 'working_hours', 'overtime'
        ])

        self.performance_reviews = pd.DataFrame(columns=[
            'review_id', 'employee_id', 'review_date', 'reviewer',
            'performance_score', 'strengths', 'areas_for_improvement',
            'goals', 'comments'
        ])

    def add_employee(self, emp_data: Dict) -> tuple[bool, str]:
        """Add a new employee to the system"""
        try:
            # Generate employee ID
            emp_data['employee_id'] = datetime.now().strftime('%Y%m%d%H%M%S')
            emp_data['status'] = 'Active'
            emp_data['last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # Add to database
            self.employees_db = pd.concat([
                self.employees_db,
                pd.DataFrame([emp_data])
            ], ignore_index=True)

            logger.info(f"Added new employee: {emp_data['first_name']} {emp_data['last_name']}")
            return True, emp_data['employee_id']

        except Exception as e:
            logger.error(f"Error adding employee: {str(e)}")
            return False, str(e)

    def record_attendance(self, emp_id: str, check_in: str = None,
                        check_out: str = None) -> tuple[bool, str]:
        """Record employee attendance"""
        try:
            if emp_id not in self.employees_db['employee_id'].values:
                return False, "Employee not found"

            current_time = datetime.now()
            check_in = check_in or current_time.strftime('%H:%M:%S')

            record = {
                'record_id': len(self.attendance_records) + 1,
                'employee_id': emp_id,
                'date': current_time.date().strftime('%Y-%m-%d'),
                'check_in': check_in,
                'check_out': check_out,
                'status': 'Present'
            }

            # Calculate working hours if check-out is provided
            if check_out:
                check_in_time = datetime.strptime(check_in, '%H:%M:%S')
                check_out_time = datetime.strptime(check_out, '%H:%M:%S')
                working_hours = (check_out_time - check_in_time).total_seconds() / 3600

                record['working_hours'] = round(working_hours, 2)
                record['overtime'] = max(0, working_hours - 8)  # Assuming 8-hour workday

            self.attendance_records = pd.concat([
                self.attendance_records,
                pd.DataFrame([record])
            ], ignore_index=True)

            return True, "Attendance recorded successfully"

        except Exception as e:
            logger.error(f"Error recording attendance: {str(e)}")
            return False, str(e)

    def add_performance_review(self, review_data: Dict) -> tuple[bool, str]:
        """Add a performance review"""
        try:
            if review_data['employee_id'] not in self.employees_db['employee_id'].values:
                return False, "Employee not found"

            review_data['review_id'] = len(self.performance_reviews) + 1
            review_data['review_date'] = datetime.now().strftime('%Y-%m-%d')

            self.performance_reviews = pd.concat([
                self.performance_reviews,
                pd.DataFrame([review_data])
            ], ignore_index=True)

            return True, "Performance review added successfully"

        except Exception as e:
            logger.error(f"Error adding performance review: {str(e)}")
            return False, str(e)

    def process_leave_request(self, request_data: Dict) -> tuple[bool, str]:
        """Process a leave request"""
        try:
            if request_data['employee_id'] not in self.employees_db['employee_id'].values:
                return False, "Employee not found"

            request_data['request_id'] = len(self.leave_requests) + 1
            request_data['status'] = 'Pending'

            self.leave_requests = pd.concat([
                self.leave_requests,
                pd.DataFrame([request_data])
            ], ignore_index=True)

            return True, "Leave request processed successfully"

        except Exception as e:
            logger.error(f"Error processing leave request: {str(e)}")
            return False, str(e)

    def generate_payroll_report(self, month: int, year: int) -> pd.DataFrame:
        """Generate monthly payroll report"""
        try:
            active_employees = self.employees_db[self.employees_db['status'] == 'Active']
            payroll_data = []

            for _, emp in active_employees.iterrows():
                # Calculate working days
                month_attendance = self.attendance_records[
                    (self.attendance_records['employee_id'] == emp['employee_id']) &
                    (pd.to_datetime(self.attendance_records['date']).dt.month == month) &
                    (pd.to_datetime(self.attendance_records['date']).dt.year == year)
                ]

                working_days = len(month_attendance)
                total_hours = month_attendance['working_hours'].sum() if 'working_hours' in month_attendance else 0
                overtime_hours = month_attendance['overtime'].sum() if 'overtime' in month_attendance else 0

                # Calculate basic salary
                daily_salary = emp['salary'] / 22  # Assuming 22 working days
                basic_salary = daily_salary * working_days
                overtime_pay = overtime_hours * (daily_salary / 8 * 1.5)  # 1.5x overtime rate

                payroll_data.append({
                    'employee_id': emp['employee_id'],
                    'name': f"{emp['first_name']} {emp['last_name']}",
                    'department': emp['department'],
                    'working_days': working_days,
                    'total_hours': total_hours,
                    'overtime_hours': overtime_hours,
                    'basic_salary': basic_salary,
                    'overtime_pay': overtime_pay,
                    'gross_salary': basic_salary + overtime_pay,
                    'net_salary': (basic_salary + overtime_pay) * 0.9  # Assuming 10% deductions
                })

            return pd.DataFrame(payroll_data)

        except Exception as e:
            logger.error(f"Error generating payroll report: {str(e)}")
            return pd.DataFrame()

# Demo function
def demo_hr_system():
    """Demo function to showcase HR system functionality"""
    # Initialize system
    hr_system = HRAutomationSystem()

    # Add sample employees
    employees = [
        {
            'first_name': 'John',
            'last_name': 'Doe',
            'email': 'john.doe@company.com',
            'department': 'IT',
            'position': 'Software Engineer',
            'hire_date': '2024-01-15',
            'salary': 75000,
            'manager': 'Jane Smith'
        },
        {
            'first_name': 'Alice',
            'last_name': 'Johnson',
            'email': 'alice.j@company.com',
            'department': 'HR',
            'position': 'HR Manager',
            'hire_date': '2023-11-01',
            'salary': 85000,
            'manager': 'Bob Wilson'
        }
    ]

    # Add employees
    for emp in employees:
        success, message = hr_system.add_employee(emp)
        print(f"Adding {emp['first_name']} {emp['last_name']}: {message}")

    # Record attendance
    for emp_id in hr_system.employees_db['employee_id'].values:
        hr_system.record_attendance(
            emp_id,
            check_in='09:00:00',
            check_out='17:00:00'
        )

    # Generate payroll report
    payroll_report = hr_system.generate_payroll_report(3, 2024)
    print("\nPayroll Report:")
    print(payroll_report)

    return hr_system

# Run the demo if this is the main program
if __name__ == "__main__":
    hr_system = demo_hr_system()

Adding John Doe: 20241109163647
Adding Alice Johnson: 20241109163647

Payroll Report:
      employee_id           name department  working_days  total_hours  \
0  20241109163647       John Doe         IT             0          0.0   
1  20241109163647  Alice Johnson         HR             0          0.0   

   overtime_hours  basic_salary  overtime_pay  gross_salary  net_salary  
0               0           0.0           0.0           0.0         0.0  
1               0           0.0           0.0           0.0         0.0  


<ipython-input-5-5df18358bdae>:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.attendance_records = pd.concat([


In [ ]:
# Run demo
hr_system = demo_hr_system()

# Or create your own implementation
hr = HRAutomationSystem()

Adding John Doe: 20241109163732
Adding Alice Johnson: 20241109163732

Payroll Report:
      employee_id           name department  working_days  total_hours  \
0  20241109163732       John Doe         IT             0          0.0   
1  20241109163732  Alice Johnson         HR             0          0.0   

   overtime_hours  basic_salary  overtime_pay  gross_salary  net_salary  
0               0           0.0           0.0           0.0         0.0  
1               0           0.0           0.0           0.0         0.0  


<ipython-input-5-5df18358bdae>:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.attendance_records = pd.concat([


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import gradio as gr
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class HRSystem:
    def __init__(self):
        self.employees = pd.DataFrame(columns=[
            'employee_id', 'first_name', 'last_name', 'email',
            'department', 'position', 'salary', 'manager'
        ])
        self.attendance = pd.DataFrame(columns=[
            'employee_id', 'date', 'check_in', 'check_out'
        ])
        self.leave_requests = pd.DataFrame(columns=[
            'employee_id', 'leave_type', 'start_date', 'end_date', 'status'
        ])

    def add_employee(self, first_name, last_name, email, department, position, salary, manager):
        try:
            # Generate employee ID
            emp_id = f"EMP{len(self.employees) + 1:04d}"

            # Create new employee record
            new_employee = {
                'employee_id': emp_id,
                'first_name': first_name,
                'last_name': last_name,
                'email': email,
                'department': department,
                'position': position,
                'salary': float(salary),
                'manager': manager
            }

            # Add to DataFrame
            self.employees = pd.concat([
                self.employees,
                pd.DataFrame([new_employee])
            ], ignore_index=True)

            return f"Employee added successfully. ID: {emp_id}"
        except Exception as e:
            logger.error(f"Error adding employee: {str(e)}")
            return f"Error adding employee: {str(e)}"

    def record_attendance(self, employee_id, check_in, check_out):
        try:
            if employee_id not in self.employees['employee_id'].values:
                return "Error: Employee ID not found"

            new_record = {
                'employee_id': employee_id,
                'date': datetime.now().strftime('%Y-%m-%d'),
                'check_in': check_in,
                'check_out': check_out
            }

            self.attendance = pd.concat([
                self.attendance,
                pd.DataFrame([new_record])
            ], ignore_index=True)

            return "Attendance recorded successfully"
        except Exception as e:
            return f"Error recording attendance: {str(e)}"

    def submit_leave(self, employee_id, leave_type, start_date, end_date):
        try:
            if employee_id not in self.employees['employee_id'].values:
                return "Error: Employee ID not found"

            new_leave = {
                'employee_id': employee_id,
                'leave_type': leave_type,
                'start_date': start_date,
                'end_date': end_date,
                'status': 'Pending'
            }

            self.leave_requests = pd.concat([
                self.leave_requests,
                pd.DataFrame([new_leave])
            ], ignore_index=True)

            return "Leave request submitted successfully"
        except Exception as e:
            return f"Error submitting leave request: {str(e)}"

    def get_employees(self):
        if len(self.employees) == 0:
            return "No employees found"
        return self.employees.to_string()

    def get_attendance(self):
        if len(self.attendance) == 0:
            return "No attendance records found"
        return self.attendance.to_string()

    def get_leave_requests(self):
        if len(self.leave_requests) == 0:
            return "No leave requests found"
        return self.leave_requests.to_string()

def create_interface():
    hr_system = HRSystem()

    with gr.Blocks(title="HR Management System") as app:
        gr.Markdown("# HR Management System")

        with gr.Tabs():
            # Employee Management Tab
            with gr.Tab("Employees"):
                gr.Markdown("## Employee Management")

                with gr.Row():
                    with gr.Column():
                        first_name = gr.Textbox(label="First Name")
                        last_name = gr.Textbox(label="Last Name")
                        email = gr.Textbox(label="Email")
                        department = gr.Dropdown(
                            choices=["IT", "HR", "Finance", "Marketing", "Operations"],
                            label="Department"
                        )
                    with gr.Column():
                        position = gr.Textbox(label="Position")
                        salary = gr.Number(label="Salary")
                        manager = gr.Textbox(label="Manager Name")

                add_btn = gr.Button("Add Employee")
                result = gr.Textbox(label="Result")

                view_btn = gr.Button("View All Employees")
                employee_list = gr.Textbox(label="Employee List")

                add_btn.click(
                    fn=hr_system.add_employee,
                    inputs=[first_name, last_name, email, department,
                           position, salary, manager],
                    outputs=result
                )

                view_btn.click(
                    fn=hr_system.get_employees,
                    outputs=employee_list
                )

            # Attendance Tab
            with gr.Tab("Attendance"):
                gr.Markdown("## Attendance Management")

                with gr.Row():
                    emp_id = gr.Textbox(label="Employee ID")
                    check_in = gr.Textbox(label="Check-in Time (HH:MM)")
                    check_out = gr.Textbox(label="Check-out Time (HH:MM)")

                record_btn = gr.Button("Record Attendance")
                attendance_result = gr.Textbox(label="Result")

                view_attendance_btn = gr.Button("View Attendance Records")
                attendance_list = gr.Textbox(label="Attendance Records")

                record_btn.click(
                    fn=hr_system.record_attendance,
                    inputs=[emp_id, check_in, check_out],
                    outputs=attendance_result
                )

                view_attendance_btn.click(
                    fn=hr_system.get_attendance,
                    outputs=attendance_list
                )

            # Leave Management Tab
            with gr.Tab("Leave"):
                gr.Markdown("## Leave Management")

                with gr.Row():
                    leave_emp_id = gr.Textbox(label="Employee ID")
                    leave_type = gr.Dropdown(
                        choices=["Vacation", "Sick Leave", "Personal Leave"],
                        label="Leave Type"
                    )
                    start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)")
                    end_date = gr.Textbox(label="End Date (YYYY-MM-DD)")

                submit_leave_btn = gr.Button("Submit Leave Request")
                leave_result = gr.Textbox(label="Result")

                view_leave_btn = gr.Button("View Leave Requests")
                leave_list = gr.Textbox(label="Leave Requests")

                submit_leave_btn.click(
                    fn=hr_system.submit_leave,
                    inputs=[leave_emp_id, leave_type, start_date, end_date],
                    outputs=leave_result
                )

                view_leave_btn.click(
                    fn=hr_system.get_leave_requests,
                    outputs=leave_list
                )

    return app

# Launch the application
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://980ba770c4d2752c6f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gradio as gr
import os
import logging
import json
from typing import Dict
import pytz

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SalaryCalculator:
    def __init__(self):
        self.basic_salary = 0
        self.allowances = {}
        self.deductions = {
            'tax': 0.1,  # 10% tax
            'insurance': 0.05,  # 5% insurance
            'pension': 0.03   # 3% pension
        }

    def calculate_gross_salary(self, basic_salary: float, allowances: Dict[str, float]) -> float:
        return basic_salary + sum(allowances.values())

    def calculate_deductions(self, gross_salary: float) -> Dict[str, float]:
        return {
            key: gross_salary * rate
            for key, rate in self.deductions.items()
        }

    def calculate_net_salary(self, gross_salary: float, deductions: Dict[str, float]) -> float:
        return gross_salary - sum(deductions.values())

class HRSystem:
    def __init__(self):
        self.employees = pd.DataFrame(columns=[
            'employee_id', 'first_name', 'last_name', 'email',
            'department', 'position', 'basic_salary', 'manager',
            'allowances', 'last_clock_in'
        ])
        self.attendance = pd.DataFrame(columns=[
            'employee_id', 'date', 'check_in', 'check_out', 'hours_worked'
        ])
        self.leave_requests = pd.DataFrame(columns=[
            'request_id', 'employee_id', 'leave_type', 'start_date',
            'end_date', 'status', 'approved_by', 'approval_date'
        ])
        self.payroll_history = pd.DataFrame(columns=[
            'payslip_id', 'employee_id', 'month', 'year', 'basic_salary',
            'allowances', 'gross_salary', 'deductions', 'net_salary',
            'generation_date'
        ])
        self.salary_calculator = SalaryCalculator()

    def add_employee(self, first_name, last_name, email, department, position,
                    basic_salary, manager, allowances_json):
        try:
            # Generate employee ID
            emp_id = f"EMP{len(self.employees) + 1:04d}"

            # Parse allowances
            try:
                allowances = json.loads(allowances_json)
            except:
                allowances = {
                    "housing": 0,
                    "transport": 0,
                    "meal": 0
                }

            # Create new employee record
            new_employee = {
                'employee_id': emp_id,
                'first_name': first_name,
                'last_name': last_name,
                'email': email,
                'department': department,
                'position': position,
                'basic_salary': float(basic_salary),
                'manager': manager,
                'allowances': json.dumps(allowances),
                'last_clock_in': None
            }

            # Add to DataFrame
            self.employees = pd.concat([
                self.employees,
                pd.DataFrame([new_employee])
            ], ignore_index=True)

            return f"Employee added successfully. ID: {emp_id}"
        except Exception as e:
            logger.error(f"Error adding employee: {str(e)}")
            return f"Error adding employee: {str(e)}"

    def clock_in(self, employee_id):
        try:
            if employee_id not in self.employees['employee_id'].values:
                return "Error: Employee ID not found"

            current_time = datetime.now()
            current_date = current_time.strftime('%Y-%m-%d')

            # Check if already clocked in today
            today_record = self.attendance[
                (self.attendance['employee_id'] == employee_id) &
                (self.attendance['date'] == current_date)
            ]

            if not today_record.empty:
                return "Already clocked in today"

            # Record clock-in
            new_record = {
                'employee_id': employee_id,
                'date': current_date,
                'check_in': current_time.strftime('%H:%M:%S'),
                'check_out': None,
                'hours_worked': 0
            }

            self.attendance = pd.concat([
                self.attendance,
                pd.DataFrame([new_record])
            ], ignore_index=True)

            # Update last clock in
            idx = self.employees['employee_id'] == employee_id
            self.employees.loc[idx, 'last_clock_in'] = current_time.strftime('%Y-%m-%d %H:%M:%S')

            return f"Clocked in successfully at {new_record['check_in']}"
        except Exception as e:
            return f"Error recording clock-in: {str(e)}"

    def clock_out(self, employee_id):
        try:
            if employee_id not in self.employees['employee_id'].values:
                return "Error: Employee ID not found"

            current_time = datetime.now()
            current_date = current_time.strftime('%Y-%m-%d')

            # Find today's record
            idx = (self.attendance['employee_id'] == employee_id) & \
                  (self.attendance['date'] == current_date) & \
                  (self.attendance['check_out'].isna())

            if not any(idx):
                return "No active clock-in found"

            # Calculate hours worked
            check_in_time = datetime.strptime(
                f"{current_date} {self.attendance.loc[idx, 'check_in'].iloc[0]}",
                '%Y-%m-%d %H:%M:%S'
            )
            hours_worked = (current_time - check_in_time).total_seconds() / 3600

            # Update record
            self.attendance.loc[idx, 'check_out'] = current_time.strftime('%H:%M:%S')
            self.attendance.loc[idx, 'hours_worked'] = round(hours_worked, 2)

            # Clear last clock in
            emp_idx = self.employees['employee_id'] == employee_id
            self.employees.loc[emp_idx, 'last_clock_in'] = None

            return f"Clocked out successfully. Hours worked: {round(hours_worked, 2)}"
        except Exception as e:
            return f"Error recording clock-out: {str(e)}"

    def submit_leave(self, employee_id, leave_type, start_date, end_date):
        try:
            if employee_id not in self.employees['employee_id'].values:
                return "Error: Employee ID not found"

            request_id = f"LEA{len(self.leave_requests) + 1:04d}"

            new_leave = {
                'request_id': request_id,
                'employee_id': employee_id,
                'leave_type': leave_type,
                'start_date': start_date,
                'end_date': end_date,
                'status': 'Pending',
                'approved_by': None,
                'approval_date': None
            }

            self.leave_requests = pd.concat([
                self.leave_requests,
                pd.DataFrame([new_leave])
            ], ignore_index=True)

            return f"Leave request submitted successfully. Request ID: {request_id}"
        except Exception as e:
            return f"Error submitting leave request: {str(e)}"

    def approve_leave(self, request_id, manager_id, approved):
        try:
            if request_id not in self.leave_requests['request_id'].values:
                return "Error: Leave request not found"

            idx = self.leave_requests['request_id'] == request_id
            current_status = self.leave_requests.loc[idx, 'status'].iloc[0]

            if current_status != 'Pending':
                return f"Cannot process: Request is already {current_status}"

            # Update leave request
            self.leave_requests.loc[idx, 'status'] = 'Approved' if approved else 'Rejected'
            self.leave_requests.loc[idx, 'approved_by'] = manager_id
            self.leave_requests.loc[idx, 'approval_date'] = datetime.now().strftime('%Y-%m-%d')

            return f"Leave request {request_id} has been {'approved' if approved else 'rejected'}"
        except Exception as e:
            return f"Error processing leave request: {str(e)}"

    def generate_payslip(self, employee_id, month, year):
        try:
            if employee_id not in self.employees['employee_id'].values:
                return "Error: Employee ID not found"

            # Get employee details
            emp_idx = self.employees['employee_id'] == employee_id
            employee = self.employees.loc[emp_idx].iloc[0]

            # Calculate worked days
            month_attendance = self.attendance[
                (self.attendance['employee_id'] == employee_id) &
                (pd.to_datetime(self.attendance['date']).dt.month == month) &
                (pd.to_datetime(self.attendance['date']).dt.year == year)
            ]

            total_hours = month_attendance['hours_worked'].sum()
            working_days = len(month_attendance)

            # Get allowances
            allowances = json.loads(employee['allowances'])

            # Calculate salary components
            basic_salary = employee['basic_salary']
            gross_salary = self.salary_calculator.calculate_gross_salary(basic_salary, allowances)
            deductions = self.salary_calculator.calculate_deductions(gross_salary)
            net_salary = self.salary_calculator.calculate_net_salary(gross_salary, deductions)

            # Generate payslip ID
            payslip_id = f"PAY{len(self.payroll_history) + 1:04d}"

            # Record payslip
            payslip = {
                'payslip_id': payslip_id,
                'employee_id': employee_id,
                'month': month,
                'year': year,
                'basic_salary': basic_salary,
                'allowances': json.dumps(allowances),
                'gross_salary': gross_salary,
                'deductions': json.dumps(deductions),
                'net_salary': net_salary,
                'generation_date': datetime.now().strftime('%Y-%m-%d')
            }

            self.payroll_history = pd.concat([
                self.payroll_history,
                pd.DataFrame([payslip])
            ], ignore_index=True)

            # Format payslip for display
            payslip_text = f"""
            PAYSLIP
            -------
            ID: {payslip_id}
            Month: {month}/{year}

            Employee: {employee['first_name']} {employee['last_name']}
            Department: {employee['department']}
            Position: {employee['position']}

            Working Days: {working_days}
            Total Hours: {total_hours:.2f}

            EARNINGS
            --------
            Basic Salary: ${basic_salary:,.2f}

            Allowances:
            {chr(10).join(f'- {k}: ${v:,.2f}' for k, v in allowances.items())}

            Gross Salary: ${gross_salary:,.2f}

            DEDUCTIONS
            ----------
            {chr(10).join(f'- {k}: ${v:,.2f}' for k, v in deductions.items())}

            NET SALARY: ${net_salary:,.2f}

            Generated on: {payslip['generation_date']}
            """

            return payslip_text
        except Exception as e:
            return f"Error generating payslip: {str(e)}"

    def get_employees(self):
        if len(self.employees) == 0:
            return "No employees found"
        return self.employees.to_string()

    def get_attendance(self):
        if len(self.attendance) == 0:
            return "No attendance records found"
        return self.attendance.to_string()

    def get_leave_requests(self):
        if len(self.leave_requests) == 0:
            return "No leave requests found"
        return self.leave_requests.to_string()

def create_interface():
    hr_system = HRSystem()

    with gr.Blocks(title="HR Management System") as app:
        gr.Markdown("# HR Management System")

        with gr.Tabs():
            # Employee Management Tab
            with gr.Tab("Employees"):
                gr.Markdown("## Employee Management")

                with gr.Row():
                    with gr.Column():
                        first_name = gr.Textbox(label="First Name")
                        last_name = gr.Textbox(label="Last Name")
                        email = gr.Textbox(label="Email")
                        department = gr.Dropdown(
                            choices=["IT", "HR", "Finance", "Marketing", "Operations"],
                            label="Department"
                        )
                    with gr.Column():
                        position = gr.Textbox(label="Position")
                        basic_salary = gr.Number(label="Basic Salary")
                        manager = gr.Textbox(label="Manager Name")
                        allowances = gr.Textbox(
                            label="Allowances (JSON)",
                            placeholder='{"housing": 1000, "transport": 500, "meal": 300}'
                        )

                add_btn = gr.Button("Add Employee")
                result = gr.Textbox(label="Result")

                view_btn = gr.Button("View All Employees")
                employee_list = gr.Textbox(label="Employee List")

            # Attendance Tab
            with gr.Tab("Attendance"):
                gr.Markdown("## Attendance Management")

                with gr.Row():
                    emp_id_attendance = gr.Textbox(label="Employee ID")

                with gr.Row():
                    clock_in_btn = gr.Button("Clock In")
                    clock_out_btn = gr.Button("Clock Out")

                attendance_result = gr.Textbox(label="Result")

                view_attendance_btn = gr.Button("View Attendance Records")
                attendance_list = gr.Textbox(label="Attendance Records")

            # Leave Management Tab
            with gr.Tab("Leave"):
                gr.Markdown("## Leave Management")

                with gr.Tab("Submit Request"):
                    with gr.Row():
                        leave_emp_id = gr.Textbox(label="Employee ID")
                        leave_type = gr.Dropdown(
                            choices=["Vacation", "Sick Leave", "Personal Leave"],
                            label="Leave Type"
                        )
                        start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)")
                        end_date = gr.Textbox(label="End Date (YYYY-MM-DD)")

                    submit_leave_btn = gr.Button("Submit Leave Request")
                    leave_result = gr.Textbox(label="Result")

                with gr.Tab("Approve Request"):
                    with gr.Row():
                        request_id = gr.Textbox

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gradio as gr
import os
import logging
import json
from typing import Dict, List, Optional, Union
import re
from dataclasses import dataclass
import pytz
from enum import Enum

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class LeaveStatus(Enum):
    PENDING = "Pending"
    APPROVED = "Approved"
    REJECTED = "Rejected"
    CANCELLED = "Cancelled"

class EmployeeStatus(Enum):
    ACTIVE = "Active"
    INACTIVE = "Inactive"
    ON_LEAVE = "On Leave"
    TERMINATED = "Terminated"

@dataclass
class EmployeeValidationRules:
    MIN_SALARY: float = 0
    MAX_SALARY: float = 1000000
    EMAIL_PATTERN: str = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    PHONE_PATTERN: str = r'^\+?1?\d{9,15}$'
    MIN_NAME_LENGTH: int = 2
    MAX_NAME_LENGTH: int = 50

class DataValidator:
    """Handles all data validation for the HR system"""

    @staticmethod
    def validate_email(email: str) -> tuple[bool, str]:
        """Validate email format"""
        if not email:
            return False, "Email is required"
        pattern = EmployeeValidationRules.EMAIL_PATTERN
        if not re.match(pattern, email):
            return False, "Invalid email format"
        return True, "Valid email"

    @staticmethod
    def validate_name(name: str, field: str) -> tuple[bool, str]:
        """Validate name fields"""
        if not name:
            return False, f"{field} is required"
        if len(name) < EmployeeValidationRules.MIN_NAME_LENGTH:
            return False, f"{field} too short"
        if len(name) > EmployeeValidationRules.MAX_NAME_LENGTH:
            return False, f"{field} too long"
        if not name.replace(" ", "").isalpha():
            return False, f"{field} should contain only letters"
        return True, f"Valid {field.lower()}"

    @staticmethod
    def validate_salary(salary: float) -> tuple[bool, str]:
        """Validate salary amount"""
        if not salary:
            return False, "Salary is required"
        if salary < EmployeeValidationRules.MIN_SALARY:
            return False, "Salary cannot be negative"
        if salary > EmployeeValidationRules.MAX_SALARY:
            return False, "Salary exceeds maximum limit"
        return True, "Valid salary"

    @staticmethod
    def validate_date(date_str: str, field: str) -> tuple[bool, str]:
        """Validate date format and value"""
        if not date_str:
            return False, f"{field} is required"
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d')
            if date.date() < datetime.now().date():
                return False, f"{field} cannot be in the past"
            return True, f"Valid {field.lower()}"
        except ValueError:
            return False, f"Invalid {field.lower()} format (use YYYY-MM-DD)"

    @staticmethod
    def validate_phone(phone: str) -> tuple[bool, str]:
        """Validate phone number format"""
        if not phone:
            return False, "Phone number is required"
        pattern = EmployeeValidationRules.PHONE_PATTERN
        if not re.match(pattern, phone):
            return False, "Invalid phone number format"
        return True, "Valid phone number"

class SalaryCalculator:
    """Handles all salary calculations"""

    def __init__(self):
        self.tax_rates = {
            50000: 0.1,   # 10% up to 50k
            100000: 0.2,  # 20% up to 100k
            float('inf'): 0.3  # 30% above 100k
        }
        self.allowance_rates = {
            'housing': 0.15,  # 15% of basic salary
            'transport': 0.10,  # 10% of basic salary
            'medical': 0.05,   # 5% of basic salary
            'meal': 0.05       # 5% of basic salary
        }
        self.deduction_rates = {
            'pension': 0.05,    # 5% pension contribution
            'insurance': 0.03   # 3% insurance
        }

    def calculate_tax(self, gross_salary: float) -> float:
        """Calculate tax based on progressive rates"""
        tax = 0
        remaining_salary = gross_salary
        previous_bracket = 0

        for bracket, rate in self.tax_rates.items():
            taxable_amount = min(remaining_salary, bracket - previous_bracket)
            tax += taxable_amount * rate
            remaining_salary -= taxable_amount
            if remaining_salary <= 0:
                break
            previous_bracket = bracket

        return round(tax, 2)

    def calculate_allowances(self, basic_salary: float) -> Dict[str, float]:
        """Calculate standard allowances based on basic salary"""
        return {
            allowance: round(basic_salary * rate, 2)
            for allowance, rate in self.allowance_rates.items()
        }

    def calculate_deductions(self, gross_salary: float) -> Dict[str, float]:
        """Calculate standard deductions"""
        return {
            deduction: round(gross_salary * rate, 2)
            for deduction, rate in self.deduction_rates.items()
        }

    def calculate_overtime(self, hours: float, hourly_rate: float) -> float:
        """Calculate overtime pay"""
        overtime_rate = 1.5  # 1.5x regular rate for overtime
        return round(hours * hourly_rate * overtime_rate, 2)

    def calculate_net_salary(self, basic_salary: float,
                           additional_allowances: Dict[str, float] = None,
                           overtime_hours: float = 0) -> Dict[str, float]:
        """Calculate complete salary breakdown"""
        # Calculate standard allowances
        standard_allowances = self.calculate_allowances(basic_salary)

        # Add additional allowances if provided
        total_allowances = standard_allowances.copy()
        if additional_allowances:
            total_allowances.update(additional_allowances)

        # Calculate gross salary
        gross_salary = basic_salary + sum(total_allowances.values())

        # Add overtime if applicable
        hourly_rate = basic_salary / (22 * 8)  # Assuming 22 working days
        overtime_pay = self.calculate_overtime(overtime_hours, hourly_rate)
        gross_salary += overtime_pay

        # Calculate deductions
        tax = self.calculate_tax(gross_salary)
        deductions = self.calculate_deductions(gross_salary)
        total_deductions = tax + sum(deductions.values())

        # Calculate net salary
        net_salary = gross_salary - total_deductions

        return {
            'basic_salary': basic_salary,
            'allowances': total_allowances,
            'overtime_pay': overtime_pay,
            'gross_salary': gross_salary,
            'tax': tax,
            'deductions': deductions,
            'total_deductions': total_deductions,
            'net_salary': net_salary
        }

class AttendanceTracker:
    """Handles attendance tracking and calculations"""

    def __init__(self):
        self.work_hours = {
            'standard_day': 8,
            'half_day': 4,
            'min_hours': 2
        }
        self.timezone = pytz.timezone('UTC')

    def calculate_hours_worked(self, check_in: datetime,
                             check_out: datetime) -> Dict[str, float]:
        """Calculate regular and overtime hours"""
        total_hours = (check_out - check_in).total_seconds() / 3600
        regular_hours = min(total_hours, self.work_hours['standard_day'])
        overtime_hours = max(0, total_hours - self.work_hours['standard_day'])

        return {
            'total_hours': round(total_hours, 2),
            'regular_hours': round(regular_hours, 2),
            'overtime_hours': round(overtime_hours, 2)
        }

    def validate_attendance(self, check_in: datetime,
                          check_out: datetime) -> tuple[bool, str]:
        """Validate attendance timestamps"""
        if check_in > check_out:
            return False, "Check-out time cannot be before check-in time"

        hours_worked = (check_out - check_in).total_seconds() / 3600
        if hours_worked < self.work_hours['min_hours']:
            return False, f"Minimum {self.work_hours['min_hours']} hours required"

        return True, "Valid attendance record"

class LeaveManager:
    """Handles leave management and calculations"""

    def __init__(self):
        self.leave_types = {
            'annual': {'limit': 24, 'paid': True},
            'sick': {'limit': 14, 'paid': True},
            'personal': {'limit': 7, 'paid': False},
            'maternity': {'limit': 90, 'paid': True},
            'paternity': {'limit': 14, 'paid': True}
        }

    def calculate_leave_days(self, start_date: datetime,
                           end_date: datetime) -> int:
        """Calculate number of leave days excluding weekends"""
        days = 0
        current_date = start_date
        while current_date <= end_date:
            if current_date.weekday() < 5:  # 0-4 represents Monday to Friday
                days += 1
            current_date += timedelta(days=1)
        return days

    def validate_leave_request(self, employee_id: str, leave_type: str,
                             start_date: datetime, end_date: datetime,
                             leave_balance: Dict) -> tuple[bool, str]:
        """Validate leave request"""
        if leave_type not in self.leave_types:
            return False, "Invalid leave type"

        if start_date > end_date:
            return False, "End date cannot be before start date"

        days_requested = self.calculate_leave_days(start_date, end_date)

        if days_requested > leave_balance.get(leave_type, 0):
            return False, "Insufficient leave balance"

        return True, "Valid leave request"

    def calculate_leave_balance(self, total_leaves: Dict[str, int],
                              used_leaves: Dict[str, int]) -> Dict[str, int]:
        """Calculate remaining leave balance"""
        balance = {}
        for leave_type, limit in self.leave_types.items():
            total = total_leaves.get(leave_type, limit['limit'])
            used = used_leaves.get(leave_type, 0)
            balance[leave_type] = total - used
        return balance

In [ ]:
class DatabaseManager:
    """Handles all database operations"""

    def __init__(self, data_dir: str = 'hr_data'):
        self.data_dir = data_dir
        self.tables = {
            'employees': pd.DataFrame(columns=[
                'employee_id', 'first_name', 'last_name', 'email',
                'phone', 'department', 'position', 'basic_salary',
                'joined_date', 'status', 'manager_id', 'allowances',
                'last_updated'
            ]),
            'attendance': pd.DataFrame(columns=[
                'record_id', 'employee_id', 'date', 'check_in',
                'check_out', 'regular_hours', 'overtime_hours',
                'status', 'notes'
            ]),
            'leave_requests': pd.DataFrame(columns=[
                'request_id', 'employee_id', 'leave_type', 'start_date',
                'end_date', 'days_requested', 'status', 'approved_by',
                'approval_date', 'comments'
            ]),
            'payroll': pd.DataFrame(columns=[
                'payroll_id', 'employee_id', 'period', 'basic_salary',
                'allowances', 'overtime_pay', 'gross_salary', 'deductions',
                'net_salary', 'payment_status', 'payment_date'
            ]),
            'departments': pd.DataFrame(columns=[
                'department_id', 'name', 'head_id', 'budget',
                'location', 'created_date'
            ]),
            'positions': pd.DataFrame(columns=[
                'position_id', 'title', 'department_id', 'salary_range_min',
                'salary_range_max', 'created_date'
            ])
        }
        self.initialize_database()

    def initialize_database(self):
        """Create data directory and load existing data"""
        try:
            if not os.path.exists(self.data_dir):
                os.makedirs(self.data_dir)

            for table_name in self.tables.keys():
                file_path = os.path.join(self.data_dir, f"{table_name}.csv")
                if os.path.exists(file_path):
                    self.tables[table_name] = pd.read_csv(file_path)
                    logger.info(f"Loaded {table_name} table")
        except Exception as e:
            logger.error(f"Error initializing database: {str(e)}")

    def save_all(self):
        """Save all tables to CSV files"""
        try:
            for table_name, df in self.tables.items():
                file_path = os.path.join(self.data_dir, f"{table_name}.csv")
                df.to_csv(file_path, index=False)
            logger.info("All tables saved successfully")
            return True
        except Exception as e:
            logger.error(f"Error saving tables: {str(e)}")
            return False

    def insert_record(self, table_name: str, record: Dict) -> tuple[bool, str]:
        """Insert a new record into specified table"""
        try:
            if table_name not in self.tables:
                return False, "Invalid table name"

            self.tables[table_name] = pd.concat([
                self.tables[table_name],
                pd.DataFrame([record])
            ], ignore_index=True)

            self.save_all()
            return True, f"Record added to {table_name}"
        except Exception as e:
            return False, f"Error inserting record: {str(e)}"

    def update_record(self, table_name: str, condition: Dict,
                     updates: Dict) -> tuple[bool, str]:
        """Update records in specified table"""
        try:
            if table_name not in self.tables:
                return False, "Invalid table name"

            # Create boolean mask for matching records
            mask = pd.Series(True, index=self.tables[table_name].index)
            for col, val in condition.items():
                mask &= self.tables[table_name][col] == val

            if not any(mask):
                return False, "No matching records found"

            # Update matching records
            for col, val in updates.items():
                self.tables[table_name].loc[mask, col] = val

            self.save_all()
            return True, f"Updated {sum(mask)} record(s)"
        except Exception as e:
            return False, f"Error updating record: {str(e)}"

    def query_records(self, table_name: str, condition: Dict = None) -> pd.DataFrame:
        """Query records from specified table"""
        try:
            if table_name not in self.tables:
                return pd.DataFrame()

            if not condition:
                return self.tables[table_name]

            # Apply conditions
            mask = pd.Series(True, index=self.tables[table_name].index)
            for col, val in condition.items():
                mask &= self.tables[table_name][col] == val

            return self.tables[table_name][mask]
        except Exception as e:
            logger.error(f"Error querying records: {str(e)}")
            return pd.DataFrame()

class HRSystem:
    """Main HR System class"""

    def __init__(self):
        self.db = DatabaseManager()
        self.validator = DataValidator()
        self.salary_calculator = SalaryCalculator()
        self.attendance_tracker = AttendanceTracker()
        self.leave_manager = LeaveManager()

    def add_employee(self, employee_data: Dict) -> tuple[bool, str]:
        """Add a new employee"""
        try:
            # Validate required fields
            validations = [
                self.validator.validate_name(employee_data.get('first_name'), 'First name'),
                self.validator.validate_name(employee_data.get('last_name'), 'Last name'),
                self.validator.validate_email(employee_data.get('email')),
                self.validator.validate_phone(employee_data.get('phone')),
                self.validator.validate_salary(employee_data.get('basic_salary'))
            ]

            # Check validation results
            for is_valid, message in validations:
                if not is_valid:
                    return False, message

            # Generate employee ID
            employee_id = f"EMP{len(self.db.tables['employees']) + 1:04d}"

            # Prepare employee record
            employee_record = {
                'employee_id': employee_id,
                **employee_data,
                'status': EmployeeStatus.ACTIVE.value,
                'joined_date': datetime.now().strftime('%Y-%m-%d'),
                'last_updated': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            # Insert record
            success, message = self.db.insert_record('employees', employee_record)
            if success:
                return True, f"Employee added successfully. ID: {employee_id}"
            return False, message

        except Exception as e:
            return False, f"Error adding employee: {str(e)}"

    def record_attendance(self, employee_id: str,
                        check_in_time: str = None) -> tuple[bool, str]:
        """Record attendance check-in/out"""
        try:
            # Validate employee
            employee = self.db.query_records('employees', {'employee_id': employee_id})
            if employee.empty:
                return False, "Employee not found"

            current_time = datetime.now()
            current_date = current_time.strftime('%Y-%m-%d')

            # Check for existing attendance record
            existing_record = self.db.query_records(
                'attendance',
                {
                    'employee_id': employee_id,
                    'date': current_date
                }
            )

            if existing_record.empty:
                # Check-in
                check_in = check_in_time or current_time.strftime('%H:%M:%S')
                record = {
                    'record_id': f"ATT{len(self.db.tables['attendance']) + 1:04d}",
                    'employee_id': employee_id,
                    'date': current_date,
                    'check_in': check_in,
                    'check_out': None,
                    'regular_hours': 0,
                    'overtime_hours': 0,
                    'status': 'Present',
                    'notes': ''
                }
                success, message = self.db.insert_record('attendance', record)
                return success, "Checked in successfully" if success else message
            else:
                # Check-out
                if pd.isna(existing_record.iloc[0]['check_out']):
                    check_in = datetime.strptime(
                        f"{current_date} {existing_record.iloc[0]['check_in']}",
                        '%Y-%m-%d %H:%M:%S'
                    )
                    hours = self.attendance_tracker.calculate_hours_worked(
                        check_in, current_time
                    )

                    success, message = self.db.update_record(
                        'attendance',
                        {'record_id': existing_record.iloc[0]['record_id']},
                        {
                            'check_out': current_time.strftime('%H:%M:%S'),
                            'regular_hours': hours['regular_hours'],
                            'overtime_hours': hours['overtime_hours']
                        }
                    )
                    return success, "Checked out successfully" if success else message
                else:
                    return False, "Already checked out for today"

        except Exception as e:
            return False, f"Error recording attendance: {str(e)}"

    def process_leave_request(self, request_data: Dict) -> tuple[bool, str]:
        """Process a leave request"""
        try:
            # Validate employee
            employee = self.db.query_records('employees',
                                          {'employee_id': request_data['employee_id']})
            if employee.empty:
                return False, "Employee not found"

            # Validate dates
            start_date = datetime.strptime(request_data['start_date'], '%Y-%m-%d')
            end_date = datetime.strptime(request_data['end_date'], '%Y-%m-%d')

            # Calculate leave days
            days_requested = self.leave_manager.calculate_leave_days(
                start_date, end_date
            )

            # Prepare leave record
            leave_record = {
                'request_id': f"LEA{len(self.db.tables['leave_requests']) + 1:04d}",
                **request_data,
                'days_requested': days_requested,
                'status': LeaveStatus.PENDING.value,
                'approved_by': None,
                'approval_date': None
            }

            # Insert record
            success, message = self.db.insert_record('leave_requests', leave_record)
            if success:
                return True, f"Leave request submitted successfully. ID: {leave_record['request_id']}"
            return False, message

        except Exception as e:
            return False, f"Error processing leave request: {str(e)}"

    def generate_payroll(self, employee_id: str, period: str) -> tuple[bool, str, Dict]:
        """Generate payroll for an employee"""
        try:
            # Get employee details
            employee = self.db.query_records('employees', {'employee_id': employee_id})
            if employee.empty:
                return False, "Employee not found", {}

            employee = employee.iloc[0]

            # Get attendance records for the period
            attendance_records = self.db.query_records(
                'attendance',
                {
                    'employee_id': employee_id,
                    'date': lambda x: x.startswith(period)
                }
            )

            # Calculate overtime hours
            overtime_hours = attendance_records['overtime_hours'].sum()

            # Calculate salary components
            salary_details = self.salary_calculator.calculate_net_salary(
                employee['basic_salary'],
                json.loads(employee['allowances']),
                overtime_hours
            )

            # Prepare payroll record
            payroll_record = {
                'payroll_id': f"PAY{len(self.db.tables['payroll']) + 1:04d}",
                'employee_id': employee_id,
                'period': period,
                **salary_details,
                'payment_status': 'Pending',
                'payment_date': None
            }

            # Insert record
            success, message = self.db.insert_record('payroll', payroll_record)
            if success:
                return True, "Payroll generated successfully", salary_details
            return False, message, {}

        except Exception as e:
            return False, f"Error generating payroll: {str(e)}", {}

In [ ]:
def create_hr_interface():
    """Create and launch the Gradio interface"""
    hr_system = HRSystem()

    def format_currency(amount: float) -> str:
        """Format amount as currency"""
        return f"${amount:,.2f}"

    def parse_json_safely(json_str: str) -> Dict:
        """Safely parse JSON string"""
        try:
            return json.loads(json_str)
        except:
            return {}

    # Employee Management Functions
    def add_new_employee(first_name, last_name, email, phone, department,
                        position, basic_salary, manager_id, allowances):
        """Add new employee through interface"""
        employee_data = {
            'first_name': first_name,
            'last_name': last_name,
            'email': email,
            'phone': phone,
            'department': department,
            'position': position,
            'basic_salary': float(basic_salary),
            'manager_id': manager_id,
            'allowances': allowances
        }
        success, message = hr_system.add_employee(employee_data)
        return message

    def view_employees(department=None):
        """View employees with optional department filter"""
        condition = {'department': department} if department else None
        employees = hr_system.db.query_records('employees', condition)
        if employees.empty:
            return "No employees found"

        # Format for display
        display_cols = [
            'employee_id', 'first_name', 'last_name', 'department',
            'position', 'status', 'joined_date'
        ]
        return employees[display_cols].to_string()

    # Attendance Functions
    def quick_clock_in(employee_id):
        """Quick clock-in function"""
        success, message = hr_system.record_attendance(employee_id)
        return message

    def quick_clock_out(employee_id):
        """Quick clock-out function"""
        success, message = hr_system.record_attendance(employee_id)
        return message

    def view_attendance(start_date=None, end_date=None, department=None):
        """View attendance records with filters"""
        condition = {}
        if start_date and end_date:
            condition['date'] = lambda x: start_date <= x <= end_date
        if department:
            employees = hr_system.db.query_records(
                'employees', {'department': department}
            )['employee_id'].tolist()
            condition['employee_id'] = lambda x: x in employees

        records = hr_system.db.query_records('attendance', condition)
        if records.empty:
            return "No attendance records found"
        return records.to_string()

    # Leave Management Functions
    def submit_leave_request(employee_id, leave_type, start_date, end_date, comments):
        """Submit leave request through interface"""
        request_data = {
            'employee_id': employee_id,
            'leave_type': leave_type,
            'start_date': start_date,
            'end_date': end_date,
            'comments': comments
        }
        success, message = hr_system.process_leave_request(request_data)
        return message

    def approve_leave_request(request_id, manager_id, decision, comments):
        """Approve/reject leave request"""
        updates = {
            'status': LeaveStatus.APPROVED.value if decision else LeaveStatus.REJECTED.value,
            'approved_by': manager_id,
            'approval_date': datetime.now().strftime('%Y-%m-%d'),
            'comments': comments
        }
        success, message = hr_system.db.update_record(
            'leave_requests',
            {'request_id': request_id},
            updates
        )
        return message

    def view_leave_requests(status=None):
        """View leave requests with optional status filter"""
        condition = {'status': status} if status else None
        requests = hr_system.db.query_records('leave_requests', condition)
        if requests.empty:
            return "No leave requests found"
        return requests.to_string()

    # Payroll Functions
    def generate_monthly_payroll(employee_id, year_month):
        """Generate monthly payroll"""
        success, message, details = hr_system.generate_payroll(employee_id, year_month)
        if not success:
            return message

        # Format payroll details for display
        payslip = f"""
        PAYROLL STATEMENT
        ----------------
        Period: {year_month}
        Employee ID: {employee_id}

        EARNINGS
        --------
        Basic Salary: {format_currency(details['basic_salary'])}

        Allowances:
        {chr(10).join(f'- {k}: {format_currency(v)}' for k, v in details['allowances'].items())}

        Overtime Pay: {format_currency(details['overtime_pay'])}
        Gross Salary: {format_currency(details['gross_salary'])}

        DEDUCTIONS
        ----------
        Tax: {format_currency(details['tax'])}
        {chr(10).join(f'- {k}: {format_currency(v)}' for k, v in details['deductions'].items())}

        Total Deductions: {format_currency(details['total_deductions'])}

        NET SALARY: {format_currency(details['net_salary'])}
        """
        return payslip

    def view_payroll_history(employee_id=None, period=None):
        """View payroll history with filters"""
        condition = {}
        if employee_id:
            condition['employee_id'] = employee_id
        if period:
            condition['period'] = period

        records = hr_system.db.query_records('payroll', condition)
        if records.empty:
            return "No payroll records found"
        return records.to_string()

    # Create Gradio Interface
    with gr.Blocks(title="HR Management System") as app:
        gr.Markdown("# HR Management System")

        with gr.Tabs():
            # Employee Management Tab
            with gr.Tab("Employee Management"):
                with gr.Tab("Add Employee"):
                    with gr.Row():
                        with gr.Column():
                            first_name = gr.Textbox(label="First Name")
                            last_name = gr.Textbox(label="Last Name")
                            email = gr.Textbox(label="Email")
                            phone = gr.Textbox(label="Phone")
                        with gr.Column():
                            department = gr.Dropdown(
                                choices=["IT", "HR", "Finance", "Marketing", "Sales", "Operations"],
                                label="Department"
                            )
                            position = gr.Textbox(label="Position")
                            basic_salary = gr.Number(label="Basic Salary")
                            manager_id = gr.Textbox(label="Manager ID")
                            allowances = gr.Textbox(
                                label="Allowances (JSON)",
                                placeholder='{"housing": 1000, "transport": 500}'
                            )

                    add_btn = gr.Button("Add Employee")
                    add_result = gr.Textbox(label="Result")

                with gr.Tab("View Employees"):
                    dept_filter = gr.Dropdown(
                        choices=["All", "IT", "HR", "Finance", "Marketing", "Sales", "Operations"],
                        label="Filter by Department",
                        value="All"
                    )
                    view_btn = gr.Button("View Employees")
                    employee_list = gr.Textbox(label="Employee List")

            # Attendance Tab
            with gr.Tab("Attendance"):
                with gr.Tab("Quick Clock In/Out"):
                    with gr.Row():
                        clock_emp_id = gr.Textbox(label="Employee ID")
                        clock_in_btn = gr.Button("Clock In")
                        clock_out_btn = gr.Button("Clock Out")
                    clock_result = gr.Textbox(label="Result")

                with gr.Tab("View Attendance"):
                    with gr.Row():
                        att_start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)")
                        att_end_date = gr.Textbox(label="End Date (YYYY-MM-DD)")
                        att_dept = gr.Dropdown(
                            choices=["All", "IT", "HR", "Finance", "Marketing", "Sales", "Operations"],
                            label="Department",
                            value="All"
                        )
                    view_att_btn = gr.Button("View Attendance")
                    attendance_list = gr.Textbox(label="Attendance Records")

            # Leave Management Tab
            with gr.Tab("Leave Management"):
                with gr.Tab("Submit Leave"):
                    with gr.Row():
                        leave_emp_id = gr.Textbox(label="Employee ID")
                        leave_type = gr.Dropdown(
                            choices=list(hr_system.leave_manager.leave_types.keys()),
                            label="Leave Type"
                        )
                        leave_start = gr.Textbox(label="Start Date (YYYY-MM-DD)")
                        leave_end = gr.Textbox(label="End Date (YYYY-MM-DD)")
                        leave_comments = gr.Textbox(label="Comments")

                    submit_leave_btn = gr.Button("Submit Request")
                    leave_result = gr.Textbox(label="Result")

                with gr.Tab("Approve Leave"):
                    with gr.Row():
                        request_id = gr.Textbox(label="Request ID")
                        manager_id = gr.Textbox(label="Manager ID")
                        decision = gr.Checkbox(label="Approve")
                        approval_comments = gr.Textbox(label="Comments")

                    approve_btn = gr.Button("Process Request")
                    approval_result = gr.Textbox(label="Result")

                with gr.Tab("View Requests"):
                    status_filter = gr.Dropdown(
                        choices=["All"] + [status.value for status in LeaveStatus],
                        label="Filter by Status",
                        value="All"
                    )
                    view_leave_btn = gr.Button("View Requests")
                    leave_list = gr.Textbox(label="Leave Requests")

            # Payroll Tab
            with gr.Tab("Payroll"):
                with gr.Tab("Generate Payroll"):
                    with gr.Row():
                        payroll_emp_id = gr.Textbox(label="Employee ID")
                        payroll_period = gr.Textbox(
                            label="Period (YYYY-MM)",
                            placeholder="2024-03"
                        )

                    generate_btn = gr.Button("Generate Payroll")
                    payroll_result = gr.Textbox(label="Payslip")

                with gr.Tab("View History"):
                    with gr.Row():
                        history_emp_id = gr.Textbox(label="Employee ID (optional)")
                        history_period = gr.Textbox(
                            label="Period (YYYY-MM, optional)",
                            placeholder="2024-03"
                        )

                    view_history_btn = gr.Button("View History")
                    payroll_history = gr.Textbox(label="Payroll History")

        # Connect functions to buttons
        add_btn.click(
            add_new_employee,
            inputs=[first_name, last_name, email, phone, department,
                   position, basic_salary, manager_id, allowances],
            outputs=add_result
        )

        view_btn.click(
            view_employees,
            inputs=[dept_filter],
            outputs=employee_list
        )

        clock_in_btn.click(
            quick_clock_in,
            inputs=[clock_emp_id],
            outputs=clock_result
        )

        clock_out_btn.click(
            quick_clock_out,
            inputs=[clock_emp_id],
            outputs=clock_result
        )

        view_att_btn.click(
            view_attendance,
            inputs=[att_start_date, att_end_date, att_dept],
            outputs=attendance_list
        )

        submit_leave_btn.click(
            submit_leave_request,
            inputs=[leave_emp_id, leave_type, leave_start, leave_end, leave_comments],
            outputs=leave_result
        )

        approve_btn.click(
            approve_leave_request,
            inputs=[request_id, manager_id, decision, approval_comments],
            outputs=approval_result
        )

        view_leave_btn.click(
            view_leave_requests,
            inputs=[status_filter],
            outputs=leave_list
        )

        generate_btn.click(
            generate_monthly_payroll,
            inputs=[payroll_emp_id, payroll_period],
            outputs=payroll_result
        )

        view_history_btn.click(
            view_payroll_history,
            inputs=[history_emp_id, history_period],
            outputs=payroll_history
        )

    return app

# Launch the application
if __name__ == "__main__":
    demo = create_hr_interface()
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0124d56e4f44a8a3c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
def generate_monthly_payroll(employee_id, year_month):
    """Generate monthly payroll"""
    try:
        # Get employee details
        employee = hr_system.db.query_records('employees', {'employee_id': employee_id})
        if employee.empty:
            return "Employee not found"

        employee = employee.iloc[0]

        # Get attendance records for the period
        attendance_records = hr_system.db.query_records(
            'attendance',
            {
                'employee_id': employee_id,
                'date': lambda x: x.startswith(year_month)
            }
        )

        # Calculate working days and overtime
        working_days = len(attendance_records)
        overtime_hours = attendance_records['overtime_hours'].sum() if 'overtime_hours' in attendance_records else 0

        # Basic salary calculations
        daily_rate = float(employee['basic_salary']) / 22  # Assuming 22 working days per month
        basic_salary = daily_rate * working_days

        # Calculate allowances
        try:
            allowances = json.loads(employee['allowances'])
        except:
            allowances = {
                "housing": basic_salary * 0.1,  # 10% of basic salary
                "transport": basic_salary * 0.05,  # 5% of basic salary
                "meal": basic_salary * 0.05  # 5% of basic salary
            }

        # Calculate overtime pay
        overtime_rate = daily_rate / 8 * 1.5  # 1.5 times hourly rate
        overtime_pay = overtime_hours * overtime_rate

        # Calculate gross salary
        gross_salary = basic_salary + sum(allowances.values()) + overtime_pay

        # Calculate deductions
        tax_rate = 0.1  # 10% tax
        pf_rate = 0.12  # 12% provident fund
        tax = gross_salary * tax_rate
        pf = basic_salary * pf_rate

        deductions = {
            "tax": tax,
            "provident_fund": pf
        }

        # Calculate net salary
        total_deductions = sum(deductions.values())
        net_salary = gross_salary - total_deductions

        # Format payroll statement
        payslip = f"""
PAYROLL STATEMENT
================
Period: {year_month}
Employee ID: {employee_id}
Name: {employee['first_name']} {employee['last_name']}
Department: {employee['department']}
Position: {employee['position']}

ATTENDANCE SUMMARY
----------------
Working Days: {working_days} days
Overtime Hours: {overtime_hours:.2f} hours

EARNINGS
--------
Basic Salary: ${basic_salary:,.2f}

Allowances:
{chr(10).join(f'- {k.title()}: ${v:,.2f}' for k, v in allowances.items())}

Overtime Pay: ${overtime_pay:,.2f}
Gross Salary: ${gross_salary:,.2f}

DEDUCTIONS
----------
{chr(10).join(f'- {k.title()}: ${v:,.2f}' for k, v in deductions.items())}

Total Deductions: ${total_deductions:,.2f}

NET SALARY: ${net_salary:,.2f}

Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

        # Save payroll record
        payroll_record = {
            'payroll_id': f"PAY{len(hr_system.db.tables['payroll']) + 1:04d}",
            'employee_id': employee_id,
            'period': year_month,
            'basic_salary': basic_salary,
            'allowances': json.dumps(allowances),
            'overtime_pay': overtime_pay,
            'gross_salary': gross_salary,
            'deductions': json.dumps(deductions),
            'net_salary': net_salary,
            'generation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        hr_system.db.insert_record('payroll', payroll_record)

        return payslip

    except Exception as e:
        logger.error(f"Error in payroll generation: {str(e)}")
        return f"Error generating payroll: {str(e)}"

def create_hr_interface():
    """Create and launch the Gradio interface"""
    with gr.Blocks(title="HR Management System") as app:
        # ... (previous interface code remains the same)

        # Update the payroll tab
        with gr.Tab("Payroll"):
            with gr.Tab("Generate Payroll"):
                with gr.Row():
                    payroll_emp_id = gr.Textbox(
                        label="Employee ID",
                        placeholder="Enter employee ID (e.g., EMP0001)"
                    )
                    payroll_period = gr.Textbox(
                        label="Period (YYYY-MM)",
                        placeholder="Enter period (e.g., 2024-03)"
                    )

                generate_btn = gr.Button("Generate Payroll")
                payroll_result = gr.Textbox(
                    label="Payslip",
                    lines=30,  # Increase number of visible lines
                    show_copy_button=True  # Add copy button
                )

            with gr.Tab("View History"):
                with gr.Row():
                    history_emp_id = gr.Textbox(
                        label="Employee ID (optional)",
                        placeholder="Enter employee ID or leave blank for all"
                    )
                    history_period = gr.Textbox(
                        label="Period (optional)",
                        placeholder="Enter period (YYYY-MM) or leave blank"
                    )

                view_history_btn = gr.Button("View History")
                payroll_history = gr.Textbox(
                    label="Payroll History",
                    lines=20,
                    show_copy_button=True
                )

        # Connect the payroll functions
        generate_btn.click(
            generate_monthly_payroll,
            inputs=[payroll_emp_id, payroll_period],
            outputs=payroll_result
        )

        view_history_btn.click(
            view_payroll_history,
            inputs=[history_emp_id, history_period],
            outputs=payroll_history
        )

    return app